In [1]:
##Unsupervised learning approach
# Features : DNA Post body

## High-level Steps :

# 1. Import the Community data
# 2. Text data preprocessing (html parsing, removing whitespaces, non-alphnumeric chars,etc)
# 3. Finding frequently occuring collocations(bigrams) in the post
# 4. Do POS tagging on the bigrams
# 5. Select bi-grams with having either of the types : 
         #('NOUN', 'NOUN' ),('NOUN', 'VERB'),('ADJ', 'NOUN'),(INTJ', 'VERB'),('NOUN','ADV'),('ADV','ADV') etc
# 6. Find the posts containing these frequenlty occuring collocations
# 7. Find the sentiment of the post around that collocation


In [163]:
import pandas as pd

from nltk.corpus import stopwords
stops = set(stopwords.words("english"))
#print(stops)
stops = stops.union(['please','arnt','many','cant','regards','hello','everyone','everybody','basically','hence','cheers','thanks','regards','would','let','thank','pls','hi','hiis','however','either','anyone','re','vs','hello'])
print("stops")
#print(stops)

# import DNAC Community data
df1 = pd.read_csv('discussions-dna_messages.csv')
print(df1.head())

from bs4 import BeautifulSoup
import re

# clean the text
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    cleantext = cleantext.replace('&nbsp;', ' ')
    cleantext = cleantext.replace('-','')
    cleantext = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^09A-Za-z. \t])|(\w+:\/\/\S+)"," ",cleantext).split()).lower()
    
    #print(cleantext.split(" "))
    cleantext = [y for y in cleantext.split(" ") if y not in stops]
    cleantext = " ".join([str(word) for word in cleantext])
    
    return cleantext

# parsing html data
def url_to_string(html):
    soup = BeautifulSoup(str(html), 'html5lib')
    for script in soup(["script", "style", 'aside']):
        script.extract()
    output_text = " ".join(re.split(r'[\n\t]+', soup.get_text()))
    return output_text

#parsing and cleaning 'body' column
df1['parsed_body'] = list(map(lambda x : cleanhtml(url_to_string(x)), list(df1['body'])))
df1['parsed_title'] = list(map(lambda x : cleanhtml(url_to_string(x)), list(df1['title'])))


df1=df1[["thread_id",'labels','tags','parsed_body','parsed_title']]
df1.head()

stops
  post_type  thread_id  message_id  parent_id  \
0  QUESTION    3823005     3823005        NaN   
1     REPLY    3823005     3828873  3823005.0   
2     REPLY    3823005     3824368  3823005.0   
3  QUESTION    3811694     3811694        NaN   
4     REPLY    3811694     3828414  3811694.0   

                                           view_href  \
0  https://community.cisco.com/t5/cisco-digital-n...   
1  https://community.cisco.com/t5/cisco-digital-n...   
2  https://community.cisco.com/t5/cisco-digital-n...   
3  https://community.cisco.com/t5/cisco-digital-n...   
4  https://community.cisco.com/t5/cisco-digital-n...   

                                               title  \
0  DNAC PnP: Converting IOS-XE Software from Inst...   
1  Re: DNAC PnP: Converting IOS-XE Software from ...   
2  Re: DNAC PnP: Converting IOS-XE Software from ...   
3                                add ip address DNAC   
4                            Re: add ip address DNAC   

                         

,thread_id,labels,tags,parsed_body,parsed_title
0,3823005,Cisco DNA Center,NaN,try provision switches via plug play. installa...,dnac pnp converting iosxe software install bun...
1,3823005,NaN,NaN,behavoir belongs new bug cisco dna center fixe...,dnac pnp converting iosxe software install bun...
2,3823005,NaN,NaN,tests seems special problem software version ....,dnac pnp converting iosxe software install bun...
3,3811694,Cisco DNA Center,NaN,setup run dnac 0gbit port up. ip address enter...,add ip address dnac
4,3811694,NaN,NaN,make sure spaces ip address entered issue come...,add ip address dnac


In [91]:
import re
import itertools
from __future__ import print_function
import pandas as pd
import datetime as dt
import numpy as np

import nltk
from nltk.corpus import stopwords
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures
from nltk.tokenize import RegexpTokenizer
import string
import scipy.sparse as sparse
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d


import spacy
import scattertext as st
from pprint import pprint
import en_core_web_sm
from spacy import displacy
nlp = spacy.load('en')
#nlp = en_core_web_sm.load()

from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures
from nltk.collocations import TrigramCollocationFinder
from nltk.metrics import TrigramAssocMeasures
#from nltk.collocations import QuadgramCollocationFinder
#from nltk.metrics import QuadgramAssocMeasures
#nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

In [6]:
'''
corpus = (st.CorpusFromPandas(df1, 
                              category_col='thread_id', 
                              text_col='parsed_body',
                              nlp = nlp)
          .build())
print(corpus)
#corpus.get_term_freq_df().to_csv('term_freqs.csv')
unigram_corpus = corpus.get_unigram_corpus()
unigram_corpus
'''

In [19]:
'''
html = st.produce_scattertext_explorer(corpus,
...          category='Light',
...          category_name='Light',
...          not_category_name='Dark',
...          width_in_pixels=1000,
...          metadata=df1['thread_id'])
'''

"\nhtml = st.produce_scattertext_explorer(corpus,\n...          category='Light',\n...          category_name='Light',\n...          not_category_name='Dark',\n...          width_in_pixels=1000,\n...          metadata=df1['thread_id'])\n"

In [60]:
# to provide serial of numbers
def gen(n):
    while True:
        yield n
        n += 1

In [61]:
def get_bigram_likelihood(statements, freq_filter=3, nbest=200):
    """
    Returns n (likelihood ratio) bi-grams from a group of documents
    :param        statements: list of strings
    :param freq_filter: filter for # of appearances in bi-gram
    :param       nbest: likelihood ratio for bi-grams
    """

    #words = list()
    #tokenize sentence into words
    #for statement in statements:
        # remove non-words
    tokenizer = RegexpTokenizer(r'\w+')
    words = tokenizer.tokenize(statements)

    bigram_measures = nltk.collocations.BigramAssocMeasures()
    bigram_finder = BigramCollocationFinder.from_words(words)

    # only bi-grams that appear n+ times
    bigram_finder.apply_freq_filter(freq_filter)

    # TODO: use custom stop words
    bigram_finder.apply_word_filter(lambda w: len(w) < 3 or w.lower() in nltk.corpus.stopwords.words('english'))

    bigram_results = bigram_finder.nbest(bigram_measures.likelihood_ratio, nbest)

    return bigram_finder.score_ngrams(bigram_measures.likelihood_ratio)

In [5]:
post_text_list = df1['parsed_body'].tolist()
post_text_list = [str(x) for x in post_text_list]
post_raw_text = ' '.join(post_text_list)

In [6]:
print(post_raw_text)

try provision switches via plug play. installation update process new switch complete switch install mode. problem since tag image install mode golden image switch always shown outdated even though actually desired os release. want another os update afterwards possible dna center currently unable update install bundle mode. details cisco dna center version 1.2.8 1.2.10switch c9500 40x update change 16.6.5 install mode process claiming process 1 select site2 select golden image onboarding template3 select switch define template variables4 start claiming5 device updated golden image os version switch configured template. inventory device list menu point provision switch shown outdated. switch installed install mode.6 switch displayed correct os status try another os update come install bundle mode. dna center distributes image switch aborts. also change boot variable .bin file dna center overrides line boot variable packages.conf. making mistake approach anyone similar experiences much l

In [7]:
# print 200 most frequent bi-grams occuring atleast 3 times
post_bigrams =get_bigram_likelihood(post_raw_text, freq_filter=3, nbest=200 )

In [8]:
post_bigrams

[(('dna', 'center'), 2553.683251180379),
 (('pnpa', 'dhcp'), 625.4635566633134),
 (('interface', 'gigabitethernet1'), 587.6672934185765),
 (('lan', 'automation'), 394.79852204938703),
 (('cpp', 'police'), 319.1476808441466),
 (('map', 'match'), 310.68368547583594),
 (('enc', 'identitymanagerpxgrid'), 304.5356767548988),
 (('system', 'cpp'), 294.88486988065347),
 (('192', '168'), 291.05814008283295),
 (('cisco', 'dna'), 286.85968539721364),
 (('hope', 'helps'), 273.92820320695864),
 (('match', 'system'), 250.35702526130348),
 (('vibha', 'jha'), 230.22671374940126),
 (('dna', 'advantage'), 225.14597043543506),
 (('maglev', 'config'), 206.71707014779258),
 (('identitymanagerpxgrid', 'utils'), 206.5278294658131),
 (('tac', 'case'), 205.5982412806969),
 (('cisco', 'enc'), 195.736557025963),
 (('make', 'sure'), 193.4994573229907),
 (('003', 'pnpa'), 179.21993726800326),
 (('self', 'signed'), 179.1919368682027),
 (('plug', 'play'), 174.4261203959956),
 (('prime', 'infrastructure'), 174.157882

In [9]:
# take 200 most frequent bi-grams occuring atleast 5 times
bigrams = get_bigram_likelihood(post_raw_text.lower(), freq_filter=5, nbest=200 )
bigram_joint = [' '.join(list(s[0])) for s in bigrams]

In [10]:
bigram_joint

['dna center',
 'pnpa dhcp',
 'interface gigabitethernet1',
 'lan automation',
 'cpp police',
 'map match',
 'enc identitymanagerpxgrid',
 'system cpp',
 '192 168',
 'cisco dna',
 'hope helps',
 'match system',
 'vibha jha',
 'dna advantage',
 'maglev config',
 'identitymanagerpxgrid utils',
 'tac case',
 'cisco enc',
 'make sure',
 '003 pnpa',
 'self signed',
 'plug play',
 'prime infrastructure',
 '123 123',
 'sales acceleration',
 'enterprise network',
 'control plane',
 'default gateway',
 'software defined',
 'dna essentials',
 'config wizard',
 'pid 285',
 'fusion router',
 'thread backtrace',
 'msg pdgfa',
 '255 255',
 'enterprise port',
 'opening timed',
 '3rd party',
 'sudo maglev',
 'rate useful',
 'discovery stopped',
 'center appliance',
 'com cisco',
 'static route',
 'mac address',
 '2091 9101',
 '374c pthread',
 '9101 4005',
 'brgn monmouth',
 'cr01 brgn',
 'iosd ngwc',
 'installation guide',
 '037 pnpa',
 'edge node',
 'static routes',
 'utils pxgridconnectionmanager',


In [11]:
def get_trigram_likelihood(statements, freq_filter=3, nbest=200):
    """
    Returns n (likelihood ratio) tri-grams from a group of documents
    :param        statements: list of strings
    :param freq_filter: filter for # of appearances in tri-gram
    :param       nbest: likelihood ratio for tri-grams
    """

    #words = list()
    #tokenize sentence into words
    #for statement in statements:
        # remove non-words
    tokenizer = RegexpTokenizer(r'\w+')
    words = tokenizer.tokenize(statements)

    trigram_measures = nltk.collocations.TrigramAssocMeasures()
    trigram_finder = TrigramCollocationFinder.from_words(words)

    # only tri-grams that appear n+ times
    trigram_finder.apply_freq_filter(freq_filter)

    # TODO: use custom stop words
    trigram_finder.apply_word_filter(lambda w: len(w) < 3 or w.lower() in nltk.corpus.stopwords.words('english'))

    trigram_results = trigram_finder.nbest(trigram_measures.likelihood_ratio, nbest)

    return trigram_finder.score_ngrams(trigram_measures.likelihood_ratio)

In [12]:
# take 200 most frequent tri-grams occuring atleast 5 times
trigrams = get_trigram_likelihood(post_raw_text.lower(), freq_filter=5, nbest=200 )
trigram_joint = [' '.join(list(s[0])) for s in trigrams]

In [13]:
trigram_joint

['cisco dna center',
 'dna center appliance',
 'dna center gui',
 'supported dna center',
 'using dna center',
 'dna center hardware',
 'dna center available',
 'dna center server',
 'dna center software',
 'dna center cisco',
 '003 pnpa dhcp',
 '037 pnpa dhcp',
 '036 pnpa dhcp',
 '834 pnpa dhcp',
 '839 pnpa dhcp',
 'pnpa dhcp op43',
 'match system cpp',
 'system cpp police',
 'map match system',
 'enc identitymanagerpxgrid utils',
 'cisco enc identitymanagerpxgrid',
 'com cisco enc',
 'enc identitymanagerpxgrid impl',
 'maglev config wizard',
 '123 123 123',
 'sudo maglev config',
 'open tac case',
 'vibha jha cisco',
 'identitymanagerpxgrid utils pxgridconnectionmanager',
 'dna advantage subscription',
 'cisco sales acceleration',
 '255 255 255',
 'jha cisco sales',
 'identitymanagerpxgrid utils pxgridconfigurationutils',
 'maglev config update',
 'plug play pnp',
 'sales acceleration center',
 'automation plug play',
 'cisco prime infrastructure',
 'self signed 245487054',
 'softwar

In [14]:
# appending the collocation type of the bigrams using POS Tagging
def check_collocation_type(doc, collocation):
    index = []
    length =len(doc)
    for i in range(length-1):
        if collocation.split(' ')[0] in [doc[i].text, doc[i].lemma_]:
            if collocation.split(' ')[1] in [doc[i+1].text, doc[i+1].lemma_]:
                index.append(i)
    if len(index)> 0:
        return [(doc[index[0]].text, doc[index[0]+1].text),(doc[index[0]].pos_, doc[index[0]+1].pos_)]
    else:
        return False

In [15]:
# appending the collocation type of the trigrams using POS Tagging
def check_trigram_type(doc, collocation):
    index = []
    length =len(doc)
    for i in range(length-2):
        if collocation.split(' ')[0] in [doc[i].text, doc[i].lemma_]:
            if collocation.split(' ')[1] in [doc[i+1].text, doc[i+1].lemma_]:
                if collocation.split(' ')[2] in [doc[i+2].text, doc[i+2].lemma_]:
                    index.append(i)
    if len(index)> 0:
        return [(doc[index[0]].text, doc[index[0]+1].text,doc[index[0]+2].text),(doc[index[0]].pos_, doc[index[0]+1].pos_,doc[index[0]+2].pos_)]
    else:
        return False

In [16]:
# selecting sentences containing given bi-grams
def select_sentence(review, collocation):
    sentence_list = review.split('.')
    selected_sentences = [sentence for sentence in sentence_list if collocation in sentence]
    
    return ' '.join(selected_sentences)

In [17]:
selected_collocations_with_type = []

# filtering the collocations with a particular type i.e. #('NOUN', 'NOUN' ),('NOUN', 'VERB'),('ADJ', 'NOUN'),
#  (INTJ', 'VERB'),('NOUN','ADV'),('ADV','ADV') ,etc
for collocation in bigram_joint:

    selected_data = [select_sentence(post.lower(), collocation) for post in post_text_list if collocation in post]

    
    g = gen(0)
    while True:
        index = next(g)
        if index >= len(selected_data):
            break
        post = selected_data[index]
        doc= nlp(post)
        check = check_collocation_type(doc, collocation)
        print(check)
        if check != False:
            if  'DET' in check[1]:
                break
            if  'NUM' in check[1]:
                break
            if check[1][1] == 'ADJ':
                break
            if check[1][0] == 'ADJ' and check[1][1] == 'ADV':
                break

            if check[1][0] == 'ADV' and check[1][1] == 'ADV':
                break
            if check[1][1] == 'ADP':
                break
            if check[1][0] == 'ADV' and check[1][1] == 'VERB':
                break
            if check[1][0] == 'ADJ' and check[1][1] == 'VERB':
                break
            if check[1][0] == 'VERB' and check[1][1] == 'ADV':
                break
            if check[1][0] == 'VERB' and check[1][1] == 'NOUN':
                break
            if check[1][0] == 'VERB' and check[1][1] == 'VERB':
                break
            
            if check[0][1] in ["dna","dnac","dna center","DNA 1.2.6","dna lan"]:
                break
            if check[0][1] == ["installation","configuration","integration","download","support","update"]:
                break
            if check[0][0] in ['issue','unable','problem','disable','first','failed','error','headache',
                               'perfect', 'bug', 'easy', 'good', 'fast','slow',"best","useful"]:
                break
            if check[0][1] in ["recommend","update","delete","leaking"]:
                break  
            

            selected_collocations_with_type.append(check)

            full_data_doc = [(token, token.ent_type_, token.pos_, token.lemma_) for token in doc]
            break
        else:   
            continue

[('dna', 'center'), ('NOUN', 'NOUN')]
[('pnpa', 'dhcp'), ('NOUN', 'NOUN')]
[('interface', 'gigabitethernet1'), ('NOUN', 'VERB')]
[('lan', 'automation'), ('NOUN', 'NOUN')]
[('cpp', 'police'), ('VERB', 'NOUN')]
[('map', 'match'), ('NOUN', 'NOUN')]
[('system', 'cpp'), ('NOUN', 'VERB')]
[('cisco', 'dna'), ('NOUN', 'NOUN')]
[('hope', 'helps'), ('NOUN', 'VERB')]
[('match', 'system'), ('NOUN', 'NOUN')]
[('vibha', 'jha'), ('VERB', 'NOUN')]
[('dna', 'advantage'), ('NOUN', 'NOUN')]
[('maglev', 'config'), ('NOUN', 'NOUN')]
[('tac', 'case'), ('NOUN', 'NOUN')]
[('make', 'sure'), ('VERB', 'ADJ')]
[('003', 'pnpa'), ('PUNCT', 'NOUN')]
[('self', 'signed'), ('NOUN', 'VERB')]
[('plug', 'play'), ('NOUN', 'NOUN')]
[('prime', 'infrastructure'), ('ADJ', 'NOUN')]
[('sales', 'acceleration'), ('NOUN', 'NOUN')]
[('enterprise', 'network'), ('NOUN', 'NOUN')]
[('control', 'plane'), ('NOUN', 'NOUN')]
[('default', 'gateway'), ('NOUN', 'NOUN')]
[('software', 'defined'), ('NOUN', 'VERB')]
[('dna', 'essentials'), ('NOUN

[('fabric', 'devices'), ('NOUN', 'NOUN')]
[('port', 'enp9s0'), ('NOUN', 'NOUN')]
[('switchport', 'port'), ('NOUN', 'NOUN')]
[('interface', 'gigabitethernet0'), ('NOUN', 'PROPN')]
[('deployment', 'guide'), ('NOUN', 'NOUN')]
[('essentials', 'advantage'), ('VERB', 'NOUN')]
[('access', 'overlay'), ('NOUN', 'NOUN')]
[('part', 'network'), ('NOUN', 'NOUN')]
[('advantage', 'licensing'), ('NOUN', 'NOUN')]
[('access', 'points'), ('NOUN', 'VERB')]
[('save', 'new'), ('VERB', 'ADJ')]
[('address', 'planning'), ('NOUN', 'VERB')]
[('failed', 'error'), ('VERB', 'NOUN')]
[('access', 'fabric'), ('NOUN', 'NOUN')]
[('something', 'like'), ('NOUN', 'ADP')]
[('network', 'essentials'), ('NOUN', 'NOUN')]
[('ise', 'base'), ('NOUN', 'NOUN')]
[('contact', 'cisco'), ('VERB', 'NOUN')]
[('non', 'sda'), ('ADJ', 'PUNCT')]
[('dnac', 'appliance'), ('VERB', 'NOUN')]
[('management', 'port'), ('NOUN', 'NOUN')]
[('non', 'cisco'), ('ADJ', 'NOUN')]
[('new', 'ios'), ('ADJ', 'ADJ')]
[('dna', 'centre'), ('NOUN', 'NOUN')]
[('integ

In [46]:
print(selected_collocations_with_type)

[[('dna', 'center'), ('NOUN', 'NOUN')], [('pnpa', 'dhcp'), ('NOUN', 'NOUN')], [('interface', 'gigabitethernet1'), ('NOUN', 'VERB')], [('lan', 'automation'), ('NOUN', 'NOUN')], [('map', 'match'), ('NOUN', 'NOUN')], [('system', 'cpp'), ('NOUN', 'VERB')], [('cisco', 'dna'), ('NOUN', 'NOUN')], [('hope', 'helps'), ('NOUN', 'VERB')], [('match', 'system'), ('NOUN', 'NOUN')], [('dna', 'advantage'), ('NOUN', 'NOUN')], [('maglev', 'config'), ('NOUN', 'NOUN')], [('tac', 'case'), ('NOUN', 'NOUN')], [('003', 'pnpa'), ('PUNCT', 'NOUN')], [('self', 'signed'), ('NOUN', 'VERB')], [('plug', 'play'), ('NOUN', 'NOUN')], [('prime', 'infrastructure'), ('ADJ', 'NOUN')], [('sales', 'acceleration'), ('NOUN', 'NOUN')], [('enterprise', 'network'), ('NOUN', 'NOUN')], [('control', 'plane'), ('NOUN', 'NOUN')], [('default', 'gateway'), ('NOUN', 'NOUN')], [('software', 'defined'), ('NOUN', 'VERB')], [('dna', 'essentials'), ('NOUN', 'NOUN')], [('config', 'wizard'), ('NOUN', 'NOUN')], [('fusion', 'router'), ('NOUN', 'N

In [18]:
selected_trigram_with_type = []

# filtering the collocations with a particular type i.e. #('NOUN', 'NOUN' ),('NOUN', 'VERB'),('ADJ', 'NOUN'),
#  (INTJ', 'VERB'),('NOUN','ADV'),('ADV','ADV') ,etc
for collocation in trigram_joint:

    selected_data = [select_sentence(post.lower(), collocation) for post in post_text_list if collocation in post]

    
    g = gen(0)
    while True:
        index = next(g)
        if index >= len(selected_data):
            break
        post = selected_data[index]
        doc= nlp(post)
        check = check_collocation_type(doc, collocation)
        print(check)
        if check != False:
            if  'DET' in check[1]:
                break
            if  'NUM' in check[1]:
                break
            if check[1][1] == 'ADJ':
                break
            if check[1][0] == 'ADJ' and check[1][1] == 'ADV':
                break

            if check[1][0] == 'ADV' and check[1][1] == 'ADV':
                break
            if check[1][1] == 'ADP':
                break
            if check[1][0] == 'ADV' and check[1][1] == 'VERB':
                break
            if check[1][0] == 'ADJ' and check[1][1] == 'VERB':
                break
            if check[1][0] == 'VERB' and check[1][1] == 'ADV':
                break
            if check[1][0] == 'VERB' and check[1][1] == 'NOUN':
                break
            if check[1][0] == 'VERB' and check[1][1] == 'VERB':
                break
            
            if check[0][1] in ["dna","dnac","dna center","DNA 1.2.6","dna lan"]:
                break
            if check[0][1] == ["installation","configuration","integration","download","support","update"]:
                break
            if check[0][0] in ['issue','unable','problem','disable','first','failed','error','headache',
                               'perfect', 'bug', 'easy', 'good', 'fast','slow',"best","useful"]:
                break
            if check[0][1] in ["recommend","update","delete","leaking"]:
                break  
            

            selected_collocations_with_type.append(check)

            full_data_doc = [(token, token.ent_type_, token.pos_, token.lemma_) for token in doc]
            break
        else:   
            continue

[('cisco', 'dna'), ('NOUN', 'NOUN')]
[('dna', 'center'), ('NOUN', 'NOUN')]
[('dna', 'center'), ('NOUN', 'NOUN')]
[('supported', 'dna'), ('VERB', 'NOUN')]
[('using', 'dna'), ('VERB', 'NOUN')]
[('dna', 'center'), ('NOUN', 'NOUN')]
[('dna', 'center'), ('VERB', 'NOUN')]
[('dna', 'center'), ('NOUN', 'NOUN')]
[('dna', 'center'), ('NOUN', 'NOUN')]
[('dna', 'center'), ('NOUN', 'NOUN')]
[('003', 'pnpa'), ('PUNCT', 'NOUN')]
[('037', 'pnpa'), ('NUM', 'DET')]
[('036', 'pnpa'), ('PUNCT', 'NOUN')]
[('834', 'pnpa'), ('NUM', 'NOUN')]
[('839', 'pnpa'), ('NUM', 'DET')]
[('pnpa', 'dhcp'), ('NOUN', 'NOUN')]
[('match', 'system'), ('NOUN', 'NOUN')]
[('system', 'cpp'), ('NOUN', 'VERB')]
[('map', 'match'), ('NOUN', 'NOUN')]
[('maglev', 'config'), ('NOUN', 'NOUN')]
[('sudo', 'maglev'), ('NOUN', 'NOUN')]
[('open', 'tac'), ('ADJ', 'NOUN')]
[('vibha', 'jha'), ('NOUN', 'NOUN')]
[('dna', 'advantage'), ('VERB', 'NOUN')]
[('cisco', 'sales'), ('NOUN', 'NOUN')]
[('jha', 'cisco'), ('NOUN', 'NOUN')]
[('maglev', 'config')

In [19]:
# creating a dictionary by grouping collocations of each type into a list
collocation_type_dict = {}
for collocation in selected_collocations_with_type:
    if collocation[1] not in collocation_type_dict:
        collocation_type_dict.update({collocation[1]:[]})
    
    collocation_type_dict[collocation[1]].append(collocation[0])
    
for c in collocation_type_dict:
    print(c," : \n", collocation_type_dict[c])
    print('\n')

('NOUN', 'NOUN')  : 
 [('dna', 'center'), ('pnpa', 'dhcp'), ('lan', 'automation'), ('map', 'match'), ('match', 'system'), ('dna', 'advantage'), ('maglev', 'config'), ('tac', 'case'), ('plug', 'play'), ('sales', 'acceleration'), ('enterprise', 'network'), ('control', 'plane'), ('default', 'gateway'), ('dna', 'essentials'), ('config', 'wizard'), ('fusion', 'router'), ('thread', 'backtrace'), ('msg', 'pdgfa'), ('enterprise', 'port'), ('sudo', 'maglev'), ('center', 'appliance'), ('mac', 'address'), ('installation', 'guides'), ('edge', 'nodes'), ('network', 'adapter'), ('msg', 'pdoon'), ('interface', 'tengigabitethernet1'), ('cisco', 'sales'), ('crypto', 'pki'), ('acceleration', 'center'), ('intra', 'cluster'), ('ngwc', 'unix'), ('c2960', 'lanlitek9'), ('sac', 'support'), ('campus', 'fabric'), ('forget', 'rate'), ('backtrace', 'traceback'), ('adjchange', 'isis'), ('aaa', 'server'), ('pnp', 'discovery'), ('cluster', 'link'), ('jha', 'cisco'), ('automation', 'plug'), ('magctl', 'service'), ('

In [20]:
selected_collocations = [x[0] for x in selected_collocations_with_type]

# select the bigrams corresponding to each collocation
bigrams_updated = [x for x in bigrams if x[0] in selected_collocations]

for b in bigrams_updated:
    print(b)

(('dna', 'center'), 2553.683251180379)
(('pnpa', 'dhcp'), 625.4635566633134)
(('interface', 'gigabitethernet1'), 587.6672934185765)
(('lan', 'automation'), 394.79852204938703)
(('map', 'match'), 310.68368547583594)
(('system', 'cpp'), 294.88486988065347)
(('hope', 'helps'), 273.92820320695864)
(('match', 'system'), 250.35702526130348)
(('vibha', 'jha'), 230.22671374940126)
(('dna', 'advantage'), 225.14597043543506)
(('maglev', 'config'), 206.71707014779258)
(('tac', 'case'), 205.5982412806969)
(('003', 'pnpa'), 179.21993726800326)
(('self', 'signed'), 179.1919368682027)
(('plug', 'play'), 174.4261203959956)
(('prime', 'infrastructure'), 174.15788227449886)
(('sales', 'acceleration'), 165.37886103078174)
(('enterprise', 'network'), 160.16814242645478)
(('control', 'plane'), 159.6958029993573)
(('default', 'gateway'), 159.06113070217833)
(('software', 'defined'), 149.34907153953014)
(('dna', 'essentials'), 148.51148819189504)
(('config', 'wizard'), 148.21236744077623)
(('fusion', 'router

In [21]:
selected_collocations_joint = [' '.join(bigram) for bigram in selected_collocations]
print(selected_collocations_joint)

['dna center', 'pnpa dhcp', 'interface gigabitethernet1', 'lan automation', 'map match', 'system cpp', 'hope helps', 'match system', 'dna advantage', 'maglev config', 'tac case', '003 pnpa', 'self signed', 'plug play', 'prime infrastructure', 'sales acceleration', 'enterprise network', 'control plane', 'default gateway', 'software defined', 'dna essentials', 'config wizard', 'fusion router', 'thread backtrace', 'msg pdgfa', 'enterprise port', 'opening timed', '3rd party', 'sudo maglev', 'discovery stopped', 'center appliance', 'static route', 'mac address', 'iosd ngwc', 'installation guides', 'edge nodes', 'static routes', 'network adapter', 'msg pdoon', 'interface tengigabitethernet1', 'open tac', 'cisco sales', 'crypto pki', 'acceleration center', 'intra cluster', 'ngwc unix', 'c2960 lanlitek9', 'sac support', 'campus fabric', 'forget rate', 'backtrace traceback', 'adjchange isis', 'window appears', 'aaa server', 'pnp discovery', 'cluster link', 'jha cisco', '036 pnpa', 'non fabric',

In [22]:
post_bigrams =get_bigram_likelihood(post_raw_text, freq_filter=3, nbest=200 )

In [23]:
# filter the post by the bi-gram
def filter_post(post, bigram):
    if bigram in post:
        return True
    else:
        return False


In [24]:
# check the sentiment polarity of the given text
def post_sentiment(string):
    sent = analyser.polarity_scores(string)
    return sent

In [26]:
final_df = pd.DataFrame({"Bigram" : [],"Post":[],"Aspects":[],"compound":[], "neg":[],"neu":[],"pos":[]})

Temp_list = []
Temp_dict = {}
for bigram in selected_collocations_joint:
        # select posts contining the bi-grams
        Filtered_list = [select_sentence(post, bigram) for post in post_text_list if filter_post(post, bigram)]
        #print(Filtered_list)
        #text=
        aspects=[]
        for s in Filtered_list:
            doc = nlp(s)
            a=[]
            for np in doc.noun_chunks:
                a.append(np.text)
            aspects.append(a)
        print(aspects)
        #print(Filtered_list)
        #print("\n")
        print(len(Filtered_list))
        print(len(aspects))
        print(len(bigram))
        if len(Filtered_list) > 0:
            df = pd.DataFrame({"Bigram":bigram,"Post":Filtered_list,"Aspects":aspects})
            #calculate the sentiment across each post for the bi-gram and calculating overall positive or negative score
            df['Sentiments'] = df['Post'].apply(lambda x: post_sentiment(x))
            
            df = pd.concat([df.drop(['Sentiments'], axis=1), df['Sentiments'].apply(pd.Series)], axis=1)
            negativity = df['neg'].sum()
            #print(negativity)
            positivity = df['pos'].sum()
            
            Temp_list.append((bigram,positivity/(positivity+negativity)))
            Temp_dict.update({bigram:positivity/(positivity+negativity)})
            final_df=pd.concat([final_df,df])
        else:
            Temp_dict.update({bigram:-1})
        
            

[['another os update', 'afterwards possible dna center', 'currently unable update install bundle mode  details', 'cisco dna center version', '1  dna center', 'image switch aborts bin file dna center overrides line boot variable packages'], ['new bug cisco dna center fixed dnac'], ['5 deleted dna center imported3'], ['cisco', 'dna center direct integration aruba', 'zero trust network component', 'looking manage', 'cisco gear', 'leverage multi vendor sdk capabilities cisco dna center platform'], ['checked compatibility sheet dna center cisco devices'], ['installation', '1 enp1s0f0 network adapter', '21gbps cisco', 'dna center gui', 'enp1s0f1 network adapter', '1 enp9s0 network adapter', '2 enp10s0 network adapter', 'cluster port default gateway', 'enp9s0 enterprise port interface'], ['poc', 'dna center ucs virtuel machine client'], ['cisco dna center hardware appliance', 'time'], [' compact switches', 'capable direct participation cisco', 'fabric fabric edge nodes cisco dna center automa

/apps/dslab/anaconda/python3/lib/python3.6/site-packages/ipykernel/__main__.py:35: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



[['980 pnpa dhcp op 43 msg pdoop', '980 pnpa dhcp op 43 msg pors', '980 pnpa dhcp op 43 msg pdokp kil pnpa', 'op43', '285 idn', 'msg', '003 pnpa dhcp op 43 msg', 'extra characters', '5a1d b2', 'i172', '003 pnpa', 'dhcp op 43 msg pdoon', '003 pnpa dhcp op 43 msg papdo', '25 003 pnpa dhcp', 'msg', 'ok process message', '14 17 25 003 xml updown pnpa', 'op43 xml interface', 'dhcp op 43 msg pdoon', '003 pnpa dhcp op 43 msg pdoop', '003 pnpa dhcp op 43 msg pdgfa', '003 pnpa dhcp op 43 msg pdgfa', '003 pnpa dhcp op 43 msg pdgfa', '003 pnpa dhcp op 43 msg pdgfa', '003 pnpa dhcp op 43 msg pdgfa', '003 pnpa dhcp op 43 msg pdoop', 'pnpa dhcp op 43 msg pors', 'pnpa dhcp op 43 msg pdokp kil pnpa dhcp', 'op43', '285 idn', '26 14 17 25 036 pnpa dhcp op 43 msg', 'pnpa dhcp', 'op 43 msg', 'extra characters', '5a1d b2', 'i172 036 pnpa dhcp op 43 msg pdoon', 'pnpa dhcp op 43 msg', '02 17 30 036 xml updown pnpa', 'op43 xml interface', '102 up 036 pnpa dhcp op 43 msg pdoon', '037 pnpa dhcp op 43 msg pdoop'

[['com profile', '1 active destination transport method http destination transport method email login success log device tracking logging theft crypto pki trustpoint sla trustpoint enrollment pkcs12 revocation check crl crypto pki trustpoint', 'tp self signed 245487054 enrollment', 'subject name cn', 'none', 'tp self signed 245487054 crypto pki certificate chain sla trustpoint certificate', '01crypto pki certificate chain', '245487054 certificate self', '01 license boot level network advantage addon dna advantage diagnostic bootup level', 'tree mode', 'pvstspanning tree', 'i', 'redundancy mode sso class map match system', 'police ewlc control description', 'ewlc controlclass map match system', 'police topology control description topology controlclass map match system', 'cpp', 'police', 'forward description', 'l2', 'lvx data packets', 'logging transit trafficclass map match system', 'default description ewlc data inter', 'fed trafficclass map match system', 'police sys data description

[['settings original setup', 'cmic launch kvm2', 'directly serverto reconfigure dnac settings', 'use', 'wizard login maglev user sudo maglev', 'nic settings'], ['running maglev configuration wizard issues connectivity'], ['use maglev', 'ssh returns', 'error'], ['sudo maglev', 'config update cli', 'wizard screens', 'vip config'], ['new dna center boot server', 'unsupported raid controller memory detected picture', 'login cimc check raid card virtual drive', 'good forced server boot', 'virtual drive bios', 'maglev configuration', 'hardware failure', 'step initial server answer'], ['changes', 'sudo maglev'], [' solution', 'enp9s0 interface', 'lan automation time dnac', 'enp9s0 ip option', '43 lan automation'], ['dnac', 'entered network config sudo maglev config update'], [' build', 'sudo maglev', 'config update cli'], [' sudo maglev', 'start screen'], [' sudo maglev', 'config update issue', 'start screen  run sudo maglev', 'wizard update experience screens'], ['configurations cat etc netw

...

[['036 pnpa dhcp op 43 msg', 'pnpa dhcp', 'op 43 msg', 'extra characters', '5a1d b2', 'i172 036 pnpa dhcp op 43 msg pdoon', 'pnpa dhcp op 43 msg', '27 02 17 30 036 pnpa dhcp op 43 msg pdoon']]
1
1
8
[['980 pnpa dhcp op 43 msg pdoop', '980 pnpa dhcp op 43 msg pors', '980 pnpa dhcp op 43 msg pdokp kil pnpa', 'op43', '285 idn', 'msg', '003 pnpa dhcp op 43 msg', 'extra characters', '5a1d b2', 'i172', '003 pnpa', 'dhcp op 43 msg pdoon', '003 pnpa dhcp op 43 msg papdo', '25 003 pnpa dhcp', 'msg', 'ok process message', '14 17 25 003 xml updown pnpa', 'op43 xml interface', 'dhcp op 43 msg pdoon', '003 pnpa dhcp op 43 msg pdoop', '003 pnpa dhcp op 43 msg pdgfa', '003 pnpa dhcp op 43 msg pdgfa', '003 pnpa dhcp op 43 msg pdgfa', '003 pnpa dhcp op 43 msg pdgfa', '003 pnpa dhcp op 43 msg pdgfa', '003 pnpa dhcp op 43 msg pdoop', 'pnpa dhcp op 43 msg pors', 'pnpa dhcp op 43 msg pdokp kil pnpa dhcp', 'op43', '285 idn', '26 14 17 25 036 pnpa dhcp op 43 msg', 'pnpa dhcp', 'op 43 msg', 'extra characters'

[['settings original setup', 'cmic launch kvm2', 'directly serverto reconfigure dnac settings', 'use', 'wizard login maglev user sudo maglev', 'nic settings'], ['running maglev configuration wizard issues connectivity'], ['use maglev', 'ssh returns', 'error'], ['sudo maglev', 'config update cli', 'wizard screens', 'vip config'], ['new dna center boot server', 'unsupported raid controller memory detected picture', 'login cimc check raid card virtual drive', 'good forced server boot', 'virtual drive bios', 'maglev configuration', 'hardware failure', 'step initial server answer'], ['changes', 'sudo maglev'], [' solution', 'enp9s0 interface', 'lan automation time dnac', 'enp9s0 ip option', '43 lan automation'], ['dnac', 'entered network config sudo maglev config update'], [' build', 'sudo maglev', 'config update cli'], [' sudo maglev', 'start screen'], [' sudo maglev', 'config update issue', 'start screen  run sudo maglev', 'wizard update experience screens'], ['configurations cat etc netw

[[' ise'], ['self signed certificate', 'certificate'], ['self signed root', 'ise pxgrid cert', 'alternative name dns', 'ip', 'dna cert', 'alternative names', 'ip import chain', 'subject cert pxgrid import new dna subject cert dna delete create', 'aaa server dna side'], ['313 crypto engine', '5 key addition key', '48 411 crypto engine', '5 key addition key', '245487054 com profile', '1 active destination transport method http destination transport method email login success log device tracking logging theft crypto pki trustpoint sla trustpoint enrollment pkcs12 revocation check crl crypto pki trustpoint', '245487054 enrollment', 'subject name cn', 'ios', 'none', '01crypto pki certificate chain', '245487054 certificate self', '01 license boot level network advantage addon dna advantage diagnostic bootup level', 'tree mode', 'pvstspanning tree', 'i', 'redundancy mode sso class map match system', 'police ewlc control description', 'ewlc controlclass map match system', 'police topology cont

[[' network adapter', '1 10 gbit port enp10s0 intra cluster link', 'recommended network adapter', 'enp1s0f0 management', '3 1 gbit port', 'enp1s0f1 cloud update connectivity', '4 10 gbit port', 'enp9s0 enterprise network', 'personalities network adapters'], [' network patch requirements dnac appliance', 'based best practice recommendations appliance management port cimc', '1gbit port enp1s0f0 management', '2 recommended ntp server ip address', 'required 2 recommended proxy server ip address', 'direct internet access', 'username password', 'enterprise network  dna center cluster cluster virtual ip address', 'connect enterprise network ip address', 'enterprise network subnet cluster', 'virtual ip', 'ip address', 'floats', 'live nodes cluster subnet service subnet address pool', '21 subnet', 'internal cluster communications conflict ip enterprise network dna', 'c appliance cimc management dnac appliance hardware recommended enterprise network interface connected enterprise network dna app

[[' issue', 'memory', 'new ios pki configuration'], [' issue', 'memory', 'new ios pki configuration jan'], [' issue', 'memory', 'new ios pki configuration jan'], [' issue', 'memory', 'new ios pki configuration nov', '22 16 03  issue', 'memory', 'new ios pki']]
4
4
11
[['certain circumstances', 'two operational hosts', 'multi host cluster', 'multi host cluster', 'two hosts', 'third single host', 'existing multi host cluster  cases', 'functionality unsupported multi host cluster', 'three hosts', 'two operational hosts software', 'backup filerestarting clusterremoving active host', 'already faulty host', 'reachability ip connectivity multi host cluster', 'guys', 'new cisco dna cluster']]
1
1
10
[['cisco dna advantage', 'policy based automation', 'cisco dna assurance'], ['access', 'network domains'], ['dna advantage licence', 'devices', 'routers', 'policy based automation', 'cisco dna assurance'], ['dna advantage licence', 'advanced features', 'basic features dna center policy', 'based aut

In [164]:
from nltk import word_tokenize, pos_tag, ne_chunk
from nltk import RegexpParser
from nltk import Tree


# Defining a grammar & Parser
grammar1 = r"""
      NP:
            {<NN>+}
            {<NNS>+}# chunk consecutive nouns
      """

grammar2= r"""NP: {<NNP>+}
             S: {<NN>+}# chunk consecutive proper nouns"""

chunker1 = RegexpParser(grammar1)

chunker2= RegexpParser(grammar2)

def get_continuous_chunks(text, chunk_func=ne_chunk):
    chunked = chunk_func(pos_tag(word_tokenize(text)))
    #print(chunked)
    continuous_chunk = []
    current_chunk = []

    for subtree in chunked:
        #print(type(subtree))
        if type(subtree) == Tree: 
            current_chunk.append(" ".join([token for token, pos in subtree.leaves()]))
        elif current_chunk:
            named_entity = " ".join(current_chunk)
            if named_entity not in continuous_chunk:
                continuous_chunk.append(named_entity)
                current_chunk = []
        else:
            continue
    if len(continuous_chunk)==0 and len(current_chunk)>0:
        continuous_chunk=current_chunk

    return continuous_chunk




#final_df["ne_chunks"]=final_df['Post'].apply(lambda sent: get_continuous_chunks(sent, chunker.parse))

In [165]:
# sentence1="When setup was run for Cisco DNA Center gbit port was not up. no ip address was entered.How can I recover from this situation"
# print(get_continuous_chunks(sentence1, chunker2.parse))
# print(get_continuous_chunks(sentence1, chunker1.parse))
# #print(df1.head())

# sentence2="hello everybody we try to provision our switches via plug and play. after the installation and update process of a new switch is complete the switch is in install mode. problem since i can not tag an image in install mode as a golden image my switch is always shown as outdated even though it is actually on the desired os release. if i want to do another os update afterwards this is not possible because the dna center is currently unable to update from install to bundle mode. further details cisco dna center version 1.2.8 1.2.10switch c9500 40x update change from 16.6.5 install mode process of the claiming process 1 select site2 select golden image onboarding template3 select switch and define template variables4 start claiming5 after that the device is updated to the golden image os version and the switch is configured with the template. in the inventory device list in the menu point provision the switch is shown as outdated. this is because the switch is installed in install mode.6 so that the switch is displayed with the correct os status i try another os update to come from the install in the bundle mode. the dna center distributes the image to the switch but then aborts. also if i change the boot variable to the .bin file the dna center overrides this line with the boot variable to packages.conf. am i making a mistake in my approach does anyone have similar experiences i very much look forward to hearing from you. best regards johannes grimm"
# pos = nltk.pos_tag(word_tokenize(sentence2))
# #print(pos)
# print(get_continuous_chunks(sentence2, chunker2.parse))
# print(get_continuous_chunks(sentence2, chunker1.parse))
# sentence3="dna lan automation timing cat 3650"
# # pos = nltk.pos_tag(word_tokenize(sentence3))
# # print(pos)
# print(get_continuous_chunks(sentence3, chunker2.parse))
# print(get_continuous_chunks(sentence3, chunker1.parse))

# sentence4="cisco dna center platform"
# pos = nltk.pos_tag(word_tokenize(sentence4))
# print(pos)
# print(get_continuous_chunks(sentence4, chunker2.parse))
# print(get_continuous_chunks(sentence4, chunker1.parse))

df1["targets_body"]=df1['parsed_body'].apply(lambda sent: get_continuous_chunks(sent, chunker2.parse))
df1["aspects_body"]=df1['parsed_body'].apply(lambda sent: get_continuous_chunks(sent, chunker1.parse))
df1["targets_title"]=df1['parsed_title'].apply(lambda sent: get_continuous_chunks(sent, chunker2.parse))
df1["aspects_title"]=df1['parsed_title'].apply(lambda sent: get_continuous_chunks(sent, chunker1.parse))


In [166]:
df1.to_csv("targets_aspects_new2.csv")

In [28]:
Temp_dict

{'dna center': 0.8572328262765236,
 'pnpa dhcp': 1.0,
 'interface gigabitethernet1': 0.9654320987654321,
 'lan automation': 0.7864529472595657,
 'map match': 0.6315789473684211,
 'system cpp': 0.6315789473684211,
 'hope helps': 1.0,
 'match system': 0.6315789473684211,
 'dna advantage': 0.9568139808435557,
 'maglev config': 0.4938181818181819,
 'tac case': 0.9298157031052764,
 '003 pnpa': 1.0,
 'self signed': 0.8813559322033898,
 'plug play': 1.0,
 'prime infrastructure': 0.9693757361601885,
 'sales acceleration': 1.0,
 'enterprise network': 0.9188567043928639,
 'control plane': 0.975609756097561,
 'default gateway': 0.739460370994941,
 'software defined': 1.0,
 'dna essentials': 0.9772964509394572,
 'config wizard': 0.04311926605504587,
 'fusion router': 0.9572327044025157,
 'thread backtrace': 0.25,
 'msg pdgfa': nan,
 'enterprise port': 1.0,
 'opening timed': 0.3056112224448898,
 '3rd party': 0.9782870928829915,
 'sudo maglev': 1.0,
 'discovery stopped': 0.26889714993804215,
 'cente

In [29]:
final_df = final_df.rename(columns={'compound': 'overall sentiment', 'neg': 'negative score','neu': 'nuetral score', 'pos':'positive score'})
print(final_df)

                                              Aspects  \
0   [another os update, afterwards possible dna ce...   
1               [new bug cisco dna center fixed dnac]   
2                    [5 deleted dna center imported3]   
3   [cisco, dna center direct integration aruba, z...   
4   [checked compatibility sheet dna center cisco ...   
5   [installation, 1 enp1s0f0 network adapter, 21g...   
6        [poc, dna center ucs virtuel machine client]   
7         [cisco dna center hardware appliance, time]   
8   [ compact switches, capable direct participati...   
9                                [release dna center]   
10       [possible dna center run hypervisors, hyper]   
11  [ contrast smallest date cisco dna center hard...   
12  [ iso, available customers, physical cisco dna...   
13      [earlier cisco dna center hardware appliance]   
14  [update roadmap information cisco dna center v...   
15  [possible install dna center vmware esxi menti...   
16  [ afraid customer, cheap se

In [73]:
final_df["np_textblob"]=final_df['Post'].apply(lambda sent: TextBlob(sent).noun_phrases)

In [74]:
# exporting results to csv file
final_df.to_csv("sentiment_textblob_ne_chunk.csv")

In [70]:
from textblob import TextBlob

In [ ]:
final_df["np_textblob"]=final_df['Post'].apply(lambda sent: TextBlob(txt).noun_phrases)

In [17]:
from collections import OrderedDict
import numpy as np
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

nlp = spacy.load('en_core_web_sm')

class TextRank4Keyword():
    """Extract keywords from text"""
    
    def __init__(self):
        self.d = 0.85 # damping coefficient, usually is .85
        self.min_diff = 1e-5 # convergence threshold
        self.steps = 10 # iteration steps
        self.node_weight = None # save keywords and its weight

    
    def set_stopwords(self, stopwords):  
        """Set stop words"""
        for word in STOP_WORDS.union(set(stopwords)):
            lexeme = nlp.vocab[word]
            lexeme.is_stop = True
    
    def sentence_segment(self, doc, candidate_pos, lower):
        """Store those words only in candidate_pos"""
        
        sentences = []
        for sent in doc.sents:
            selected_words = []
            for token in sent:
                # Store words only with cadidate POS tag
                print("token :"+ token.text +"  POS Tag : "+str(token.tag_))
                if token.pos_ in candidate_pos and token.is_stop is False:
                    if lower is True:
                        selected_words.append(token.text.lower())
                    else:
                        selected_words.append(token.text)
            sentences.append(selected_words)
        return sentences
        
    def get_vocab(self, sentences):
        """Get all tokens"""
        vocab = OrderedDict()
        i = 0
        for sentence in sentences:
            for word in sentence:
                if word not in vocab:
                    vocab[word] = i
                    i += 1
        return vocab
    
    def get_token_pairs(self, window_size, sentences):
        """Build token_pairs from windows in sentences"""
        token_pairs = list()
        for sentence in sentences:
            for i, word in enumerate(sentence):
                for j in range(i+1, i+window_size):
                    if j >= len(sentence):
                        break
                    pair = (word, sentence[j])
                    if pair not in token_pairs:
                        token_pairs.append(pair)
        return token_pairs
        
    def symmetrize(self, a):
        return a + a.T - np.diag(a.diagonal())
    
    def get_matrix(self, vocab, token_pairs):
        """Get normalized matrix"""
        # Build matrix
        vocab_size = len(vocab)
        g = np.zeros((vocab_size, vocab_size), dtype='float')
        for word1, word2 in token_pairs:
            i, j = vocab[word1], vocab[word2]
            g[i][j] = 1
            
        # Get Symmeric matrix
        g = self.symmetrize(g)
        
        # Normalize matrix by column
        norm = np.sum(g, axis=0)
        g_norm = np.divide(g, norm, where=norm!=0) # this is ignore the 0 element in norm
        
        return g_norm

    
    def get_keywords(self, number=10):
        """Print top number keywords"""
        node_weight = OrderedDict(sorted(self.node_weight.items(), key=lambda t: t[1], reverse=True))
        for i, (key, value) in enumerate(node_weight.items()):
            #print(key + ' - ' + str(value))
            if i > number:
                break
        return node_weight
        
        
    def analyze(self, text, 
                candidate_pos=['NOUN', 'PROPN'], 
                window_size=4, lower=False, stopwords=list()):
        """Main function to analyze text"""
        print("inside analyze")
        
        # Set stop words
        self.set_stopwords(stopwords)
        
        # Pare text by spaCy
        doc = nlp(text)
        
        # Filter sentences
        sentences = self.sentence_segment(doc, candidate_pos, lower) # list of list of words
        
        # Build vocabulary
        vocab = self.get_vocab(sentences)
        
        # Get token_pairs from windows
        token_pairs = self.get_token_pairs(window_size, sentences)
        
        # Get normalized matrix
        g = self.get_matrix(vocab, token_pairs)
        
        # Initionlization for weight(pagerank value)
        pr = np.array([1] * len(vocab))
        
        # Iteration
        previous_pr = 0
        for epoch in range(self.steps):
            pr = (1-self.d) + self.d * np.dot(g, pr)
            if abs(previous_pr - sum(pr))  < self.min_diff:
                break
            else:
                previous_pr = sum(pr)

        # Get weight for each node
        node_weight = dict()
        for word, index in vocab.items():
            node_weight[word] = pr[index]
        
        self.node_weight = node_weight

In [18]:
def get_keywords(text):
    tr4w.analyze(text, candidate_pos = ['NOUN', 'PROPN'], window_size=5, lower=False)
    l=list(tr4w.get_keywords(10).keys())
    return l

In [37]:
tr4w = TextRank4Keyword()

#print(df1["parsed_body"].head())
#df1["keywords"]=df1["parsed_body"].apply(lambda x : get_keywords(x))

In [38]:
text='When setup was run for Cisco DNA Center gbit port was not up. no ip address was entered.How can I recover from this situation '

print(get_keywords(text))

inside analyze
token :When  POS Tag : WRB
token :setup  POS Tag : NN
token :was  POS Tag : VBD
token :run  POS Tag : VBN
token :for  POS Tag : IN
token :Cisco  POS Tag : NNP
token :DNA  POS Tag : NNP
token :Center  POS Tag : NNP
token :gbit  POS Tag : NN
token :port  POS Tag : NN
token :was  POS Tag : VBD
token :not  POS Tag : RB
token :up  POS Tag : IN
token :.  POS Tag : .
token :no  POS Tag : DT
token :ip  POS Tag : NN
token :address  POS Tag : NN
token :was  POS Tag : VBD
token :entered  POS Tag : VBN
token :.  POS Tag : .
token :How  POS Tag : WRB
token :can  POS Tag : MD
token :I  POS Tag : PRP
token :recover  POS Tag : VB
token :from  POS Tag : IN
token :this  POS Tag : DT
token :situation  POS Tag : NN
['Cisco', 'DNA', 'Center', 'gbit', 'ip', 'address', 'setup', 'port', 'situation']


In [6]:
import spacy
from textpipeliner import PipelineEngine, Context
from textpipeliner.pipes import *

pipes_structure = [SequencePipe([FindTokensPipe("VERB/nsubj/NNP"),
                                 NamedEntityFilterPipe(),
                                 NamedEntityExtractorPipe()]),
                       AggregatePipe([FindTokensPipe("VERB"),
                                      FindTokensPipe("VERB/xcomp/VERB/aux/*"),
                                      FindTokensPipe("VERB/xcomp/VERB")]),
                       AnyPipe([FindTokensPipe("VERB/[acomp,amod]/ADJ"),
                                AggregatePipe([FindTokensPipe("VERB/[dobj,attr]/NOUN/det/DET"),
                                               FindTokensPipe("VERB/[dobj,attr]/NOUN/[acomp,amod]/ADJ")])])
                      ]

text='When setup was run for Cisco DNA Center gbit port was not up. no ip address was entered.How can I recover from this situation '
doc = nlp(text)
print(doc)
engine = PipelineEngine(pipes_structure, Context(doc), [0,1,2])
engine.process()

When setup was run for Cisco DNA Center gbit port was not up. no ip address was entered.How can I recover from this situation 


[]

In [15]:
from nltk.stem.wordnet import WordNetLemmatizer
from spacy.lang.en import English

SUBJECTS = ["nsubj", "nsubjpass", "csubj", "csubjpass", "agent", "expl"]
OBJECTS = ["dobj", "dative", "attr", "oprd"]
ADJECTIVES = ["acomp", "advcl", "advmod", "amod", "appos", "nn", "nmod", "ccomp", "complm",
              "hmod", "infmod", "xcomp", "rcmod", "poss"," possessive"]
COMPOUNDS = ["compound"]
PREPOSITIONS = ["prep"]

def getSubsFromConjunctions(subs):
    moreSubs = []
    for sub in subs:
        # rights is a generator
        rights = list(sub.rights)
        rightDeps = {tok.lower_ for tok in rights}
        if "and" in rightDeps:
            moreSubs.extend([tok for tok in rights if tok.dep_ in SUBJECTS or tok.pos_ == "NOUN"])
            if len(moreSubs) > 0:
                moreSubs.extend(getSubsFromConjunctions(moreSubs))
    return moreSubs

def getObjsFromConjunctions(objs):
    moreObjs = []
    for obj in objs:
        # rights is a generator
        rights = list(obj.rights)
        rightDeps = {tok.lower_ for tok in rights}
        if "and" in rightDeps:
            moreObjs.extend([tok for tok in rights if tok.dep_ in OBJECTS or tok.pos_ == "NOUN"])
            if len(moreObjs) > 0:
                moreObjs.extend(getObjsFromConjunctions(moreObjs))
    return moreObjs

def getVerbsFromConjunctions(verbs):
    moreVerbs = []
    for verb in verbs:
        rightDeps = {tok.lower_ for tok in verb.rights}
        if "and" in rightDeps:
            moreVerbs.extend([tok for tok in verb.rights if tok.pos_ == "VERB"])
            if len(moreVerbs) > 0:
                moreVerbs.extend(getVerbsFromConjunctions(moreVerbs))
    return moreVerbs

def findSubs(tok):
    head = tok.head
    while head.pos_ != "VERB" and head.pos_ != "NOUN" and head.head != head:
        head = head.head
    if head.pos_ == "VERB":
        subs = [tok for tok in head.lefts if tok.dep_ == "SUB"]
        if len(subs) > 0:
            verbNegated = isNegated(head)
            subs.extend(getSubsFromConjunctions(subs))
            return subs, verbNegated
        elif head.head != head:
            return findSubs(head)
    elif head.pos_ == "NOUN":
        return [head], isNegated(tok)
    return [], False

def isNegated(tok):
    negations = {"no", "not", "n't", "never", "none"}
    for dep in list(tok.lefts) + list(tok.rights):
        if dep.lower_ in negations:
            return True
    return False

def findSVs(tokens):
    svs = []
    verbs = [tok for tok in tokens if tok.pos_ == "VERB"]
    for v in verbs:
        subs, verbNegated = getAllSubs(v)
        if len(subs) > 0:
            for sub in subs:
                svs.append((sub.orth_, "!" + v.orth_ if verbNegated else v.orth_))
    return svs

def getObjsFromPrepositions(deps):
    objs = []
    for dep in deps:
        if dep.pos_ == "ADP" and dep.dep_ == "prep":
            objs.extend([tok for tok in dep.rights if tok.dep_  in OBJECTS or (tok.pos_ == "PRON" and tok.lower_ == "me")])
    return objs

def getAdjectives(toks):
    toks_with_adjectives = []
    for tok in toks:
        adjs = [left for left in tok.lefts if left.dep_ in ADJECTIVES]
        adjs.append(tok)
        adjs.extend([right for right in tok.rights if tok.dep_ in ADJECTIVES])
        tok_with_adj = " ".join([adj.lower_ for adj in adjs])
        toks_with_adjectives.extend(adjs)

    return toks_with_adjectives

def getObjsFromAttrs(deps):
    for dep in deps:
        if dep.pos_ == "NOUN" and dep.dep_ == "attr":
            verbs = [tok for tok in dep.rights if tok.pos_ == "VERB"]
            if len(verbs) > 0:
                for v in verbs:
                    rights = list(v.rights)
                    objs = [tok for tok in rights if tok.dep_ in OBJECTS]
                    objs.extend(getObjsFromPrepositions(rights))
                    if len(objs) > 0:
                        return v, objs
    return None, None

def getObjFromXComp(deps):
    for dep in deps:
        if dep.pos_ == "VERB" and dep.dep_ == "xcomp":
            v = dep
            rights = list(v.rights)
            objs = [tok for tok in rights if tok.dep_ in OBJECTS]
            objs.extend(getObjsFromPrepositions(rights))
            if len(objs) > 0:
                return v, objs
    return None, None

def getAllSubs(v):
    verbNegated = isNegated(v)
    '''
    print("verb "+v.text)
    print("lefts :")
    for tok in v.lefts:
        print("token :"+tok.text)
        print(tok.dep_)
        print(tok.pos_)
    '''
    neighbours=[]
    index=v.i
    
    for i in range(-3,0):
        
        try:
            neighbours.extend([v.nbor(i)])
        except:
           pass 
    print(neighbours)
        
    subs = [tok for tok in neighbours if tok.dep_ in SUBJECTS and tok.pos_ != "DET" or tok.pos == "NOUN"]
    ##print("verb :"+v.text)
    '''
    print("subs earlier:")
    print(subs)
    print("\n")
    '''
    if len(subs) > 0:
        subs.extend(getSubsFromConjunctions(subs))
    else:
        foundSubs, verbNegated = findSubs(v)
        subs.extend(foundSubs)
    '''
    print("subs after:")
    '''
    print(subs)
    return subs, verbNegated

def getAllObjs(v):
    # rights is a generator
    rights = list(v.rights)
    objs = [tok for tok in rights if tok.dep_ in OBJECTS]
    objs.extend(getObjsFromPrepositions(rights))

    potentialNewVerb, potentialNewObjs = getObjFromXComp(rights)
    if potentialNewVerb is not None and potentialNewObjs is not None and len(potentialNewObjs) > 0:
        objs.extend(potentialNewObjs)
        v = potentialNewVerb
    if len(objs) > 0:
        objs.extend(getObjsFromConjunctions(objs))
    return v, objs

def getAllObjsWithAdjectives(v):
    # rights is a generator
    rights = list(v.rights)
   
    print("verb :")
    print(v)
    '''
    print("rights :")
    print(rights)
    '''
    r_neighbour=[]
    for i in range(1,4):
        
        
        try:
            if v.nbor(i).text !=".":
                r_neighbour.extend([v.nbor(i)])
        except:
           pass 
    print("neighbours :")
    print(r_neighbour)
    objs = [tok for tok in r_neighbour if tok.dep_ in OBJECTS]
    print("objs 1:")
    print(objs)
    if len(objs)== 0:
        objs = [tok for tok in r_neighbour if tok.dep_ in ADJECTIVES]

    objs.extend(getObjsFromPrepositions(rights))

    potentialNewVerb, potentialNewObjs = getObjFromXComp(rights)
    if potentialNewVerb is not None and potentialNewObjs is not None and len(potentialNewObjs) > 0:
        objs.extend(potentialNewObjs)
        v = potentialNewVerb
    if len(objs) > 0:
        objs.extend(getObjsFromConjunctions(objs))
    print("objs 2:")
    print(objs)
    return v, objs

def findSVOs(tokens):
    svos = []
    verbs = [tok for tok in tokens if tok.pos_ == "VERB" and tok.dep_ != "aux"]
    for v in verbs:
        subs, verbNegated = getAllSubs(v)
        print(subs+" "+ verbNegated)
        # hopefully there are subs, if not, don't examine this verb any longer
        if len(subs) > 0:
            v, objs = getAllObjs(v)
            for sub in subs:
                for obj in objs:
                    objNegated = isNegated(obj)
                    svos.append((sub.lower_, "!" + v.lower_ if verbNegated or objNegated else v.lower_, obj.lower_))
    return svos

def findSVAOs(tokens):
    #print("inside")
    svos = []
    for tok in tokens:
        print("token :"+tok.text)
        print(tok.dep_)
        print(tok.pos_)
        print("\n")
    verbs = [tok for tok in tokens if tok.pos_ == "VERB" and tok.dep_ != "aux"]
    print(verbs)
    for v in verbs:
        subs, verbNegated = getAllSubs(v)
        # hopefully there are subs, if not, don't examine this verb any longer
        if len(subs) > 0:
            v, objs = getAllObjsWithAdjectives(v)
            for sub in subs:
                for obj in objs:
                    objNegated = isNegated(obj)
                    obj_desc_tokens = generate_left_right_adjectives(obj)
                    sub_compound = generate_sub_compound(sub)
                    svos.append((" ".join(tok.lower_ for tok in sub_compound), "!" + v.lower_ if verbNegated or objNegated else v.lower_, " ".join(tok.lower_ for tok in obj_desc_tokens)))
    return svos

def generate_sub_compound(sub):
    sub_compunds = []
    for tok in sub.lefts:
        if tok.dep_ in COMPOUNDS:
            sub_compunds.extend(generate_sub_compound(tok))
    sub_compunds.append(sub)
    for tok in sub.rights:
        if tok.dep_ in COMPOUNDS:
            sub_compunds.extend(generate_sub_compound(tok))
    return sub_compunds

def generate_left_right_adjectives(obj):
    obj_desc_tokens = []
    for tok in obj.lefts:
        if tok.dep_ in ADJECTIVES:
            obj_desc_tokens.extend(generate_left_right_adjectives(tok))
    obj_desc_tokens.append(obj)

    for tok in obj.rights:
        if tok.dep_ in ADJECTIVES:
            obj_desc_tokens.extend(generate_left_right_adjectives(tok))

    return obj_desc_tokens

def getAspects(tokens):
    svos = []
    for tok in tokens:
        print("token :"+tok.text)
        print(tok.dep_)
        print(tok.pos_)
        print("\n")
    verbs = [tok for tok in tokens if tok.pos_ == "VERB" and tok.dep_ != "aux"]
    print(verbs)
    for v in verbs:
        subs, verbNegated = getAllSubs(v)
        svos.extend(subs)
    return (svos)

def mergeTokens(tokens):
    #merged_tokens=[]
    for tok in tokens[1:]:
        try:
            if tok.pos=="NOUN" and tok.nbor(-1)=="NOUN":
                span=tokens[tok.i:tok.i+1]
                span.merge()
                merged_token.append(tok)
        except:
            pass
    return tokens
    
    

In [16]:
import spacy
parser = spacy.load('en', disable=['ner','textcat'])

#sentence = "Donald Trump is the worst president of USA, but Hillary is better than him"
sentence=""
parse = parser(sentence)
#print(parse)
print(findSVAOs(parse))
#print(getAspects(parse))
print(mergeTokens)

When setup was run for Cisco DNA Center gbit port was not up. no ip address was entered.How can I recover from this situation
<function mergeTokens at 0x7fb6b1c6e378>
